In [ ]:
import gradio as gr
import os
demo = None #added to allow restart

def start_game(main_loop, share=False):
    # added code to support restart
    global demo
    # If demo is already running, close it first
    if demo is not None:
        demo.close()

    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250, placeholder="Type 'start game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="Bird Adventure Game",
        # description="Ask Yes Man any question",
        theme="soft",
        examples=["Look around", "Continue the story"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear",
                           )
    demo.launch(share=share, server_name="0.0.0.0")

def test_main_loop(message, history):
    return 'Entered Action: ' + message

start_game(test_main_loop)

# Generating an Initial Start

In [ ]:
from helper import load_world, save_world
from together import Together
from helper import get_together_api_key, load_env

client = Together(api_key=get_together_api_key())

habitat = load_world('../shared_data/habitat_information.json')
region = habitat['regions']

In [ ]:
system_prompt = """
You are an AI Game Master. Your job is to create the beginning of an adventure based on the habitat, region, and character a player is playing as.
Instructions:
- Use 2-4 sentences.
- Write in second person. For example: "You are Jack."
- Write in present tense. For example: "You stand at..."
- First, describe the character and their backstory.
- Then, describe where they start and what they see around them.
- Do NOT mention the species of Birdy
"""

habitat_info = f"""
Habitat: {habitat}
Region: {region}
NPCs: {region["The Larch Forests of the Taiga"]["animals"]}
Your_Character: 
Birdy is a Slender-billed Curlew, a rare and solitary bird known for its distinctive curved beak and graceful flight. You have always been a wanderer, drawn to distant places and the mysteries they hold. Though your journey is often solitary, your keen instincts and unwavering resolve guide you through even the most unfamiliar terrain.
However, you do not know your species, nor do you know that your species will become extinct.
"""


In [ ]:
model_output = client.chat.completions.create(
    model="meta-llama/Llama-3-70b-chat-hf",
    temperature=1.0,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": habitat_info + '\nYour Start:'}
    ],
)

In [ ]:
start = model_output.choices[0].message.content
print(start)
habitat['start'] = start

save_world(habitat, '../shared_data/habitat_information.json')

### Sample response I got

You are Birdy, a wanderer with a keen instinct and unwavering resolve, drawn to distant places and mysteries. You have a distinctive curved beak and a nature that's adapted to navigating unfamiliar terrain. You stand atop a snow-covered larch tree in the heart of The Larch Forests of the Taiga, your mottled brown plumage blending seamlessly into the forest's dappled shadows. The air is crisp and still, with only the occasional rustle of small mammals or the soft hooting of owls breaking the silence. The forest floor, carpeted with moss and lichen, stretches out before you, quiet and still.

# Update NPC state

In [ ]:
system_prompt = """You are an AI Game Assistant. \
Your job is to detect updates to the NPCs a player (the bird) has interacted with \
based on the most recent story and game state. \
For each interaction, update the NPC entry with a brief summary of the relationship \
and note any significant changes in their behavior or dynamics.

Given the player name, the current list of NPC interactions, and the latest story, \
return a JSON update of the NPCs in the following form:

NPC Updates:
{
    "npcUpdates": [
        {
            "name": <NPC NAME>, 
            "relationship_summary": <BRIEF SUMMARY OF RELATIONSHIP>,
            "status_change": <DESCRIPTION OF ANY SIGNIFICANT CHANGE, IF APPLICABLE>
        }...
    ]
}

- If no NPCs were interacted with, return {"npcUpdates": []}.
- Keep relationship summaries concise and factual.
- Only include NPCs clearly mentioned in the most recent story.
- Do not include information that was not explicitly described in the story.

Response must be in valid JSON.
"""


In [ ]:
def detect_npc_changes(game_state, output):
    npc_interactions = game_state['npc_interactions']
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f'Current NPC Interactions: {str(npc_interactions)}'},
        {"role": "user", "content": f'Recent Story: {output}'},
        {"role": "user", "content": 'NPC Updates'}
    ]
    chat_completion = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        temperature=0.0,
        messages=messages
    )
    response = chat_completion.choices[0].message.content
    result = json.loads(response)
    return result['npcUpdates']

from helper import get_game_state

# Example game state
game_state = get_game_state()
game_state['npc_interactions'] = {
    "Friendly Squirrel": {
        "relationship_summary": "Helped you find food near Region 1.",
        "status_change": None
    }
}

# Example usage
result = detect_npc_changes(game_state, 
"You encountered the Friendly Squirrel again, but it seems wary of you this time.")

print(result)

In [ ]:

def update_npc_interactions(npc_interactions, npc_updates):
    update_msg = ''
    
    for update in npc_updates:
        name = update['name']
        relationship_summary = update.get('relationship_summary', '')
        status_change = update.get('status_change', None)
        
        if name not in npc_interactions:
            npc_interactions[name] = {
                "relationship_summary": relationship_summary,
                "status_change": status_change
            }
            update_msg += f'\nNPC Added: {name} - {relationship_summary}'
        else:
            npc_interactions[name]["relationship_summary"] = relationship_summary
            npc_interactions[name]["status_change"] = status_change
            update_msg += f'\nNPC Updated: {name} - {relationship_summary}'
            
    return update_msg

# Creating the Main Action Loop¶

In [ ]:
def run_action(message, history, game_state):
    
    if message == 'start game':
        return game_state['start']

    system_prompt = """You are an AI Game Master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
- Write only 1-3 sentences in response. \
- Always write in second person present tense. \
For example: "You look north and see..." """

    habitat_info = f"""
Habitat: {game_state['habitat']}
Region: {game_state['region']}
NPCs: {game_state['NPCs']}
"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": habitat_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result


In [ ]:
game_state = get_game_state(npc_interactions={})

In [ ]:

def main_loop(message, history):
    output = run_action(message, history, game_state)

    # Detect changes in NPC interactions
    npc_updates = detect_npc_changes(game_state, output)
    
    # Update the NPC interactions in the game state
    update_msg = update_npc_interactions(
        game_state['npc_interactions'], 
        npc_updates
    )
    output += update_msg

    return output


# Launch the Game !
start_game(main_loop, True)